<a href="https://colab.research.google.com/github/ChessEngineUS/hybrid-quantum-protein-folding/blob/main/notebooks/quantum_circuits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚛️ Quantum Circuit Analysis

Explore the quantum computing aspects of the hybrid model.

**Topics covered:**
- 🔬 VQE circuit construction
- 📊 Hamiltonian visualization
- ⚡ Energy optimization
- 🎯 Parameter-shift gradients
- 📈 Convergence analysis

In [ ]:
import os
if os.path.exists('hybrid-quantum-protein-folding'):
    !rm -rf hybrid-quantum-protein-folding
!git clone -q https://github.com/ChessEngineUS/hybrid-quantum-protein-folding.git
%cd hybrid-quantum-protein-folding
!pip install -q -r requirements-colab.txt
!pip install -q -e .
print('✅ Setup complete!')

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram, circuit_drawer

os.chdir('/content/hybrid-quantum-protein-folding')
from hqpf.models.vqe_solver import VQESolver

print('✅ Imports complete')

## 1. VQE Circuit Construction

Build and visualize a VQE ansatz circuit.

In [ ]:
# Create VQE solver
n_qubits = 4
solver = VQESolver(
    n_qubits=n_qubits,
    ansatz_type='hardware_efficient',
    ansatz_depth=2,
    backend='statevector_simulator'
)

print(f'⚛️ VQE Solver: {n_qubits} qubits, depth 2')
print(f'Parameters: {len(solver.theta)} trainable')
print(f'Backend: {solver.backend_name}')

# Build example circuit
print('\n🔨 Building quantum circuit...')
qc = QuantumCircuit(n_qubits)

# Example: hardware-efficient ansatz
theta_vals = solver.theta.detach().numpy()
for layer in range(2):
    # Single-qubit rotations
    for i in range(n_qubits):
        qc.ry(0.5, i)
        qc.rz(0.3, i)
    # Entanglement
    for i in range(n_qubits-1):
        qc.cx(i, i+1)

print('✅ Circuit built')
print(f'Depth: {qc.depth()}')
print(f'Gates: {qc.count_ops()}')

# Visualize
print('\n📊 Circuit diagram:')
print(qc)

## 2. Hamiltonian Analysis

Construct and analyze the energy Hamiltonian.

In [ ]:
# Generate test coordinates
coords = torch.randn(n_qubits, 3)

# Construct Hamiltonian
H = solver._construct_hamiltonian(coords)
H_np = H.detach().numpy()

print(f'Hamiltonian shape: {H.shape}')
print(f'Hermitian: {torch.allclose(H, H.T)}')

# Eigenvalue analysis
eigenvalues = np.linalg.eigvalsh(H_np)
print(f'\nEigenvalue spectrum:')
print(f'  Ground state: {eigenvalues[0]:.4f}')
print(f'  Excited states: {eigenvalues[1]:.4f} to {eigenvalues[-1]:.4f}')
print(f'  Gap: {eigenvalues[1] - eigenvalues[0]:.4f}')

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Hamiltonian heatmap
im = axes[0].imshow(H_np, cmap='RdBu_r', aspect='auto')
plt.colorbar(im, ax=axes[0])
axes[0].set_title('Hamiltonian Matrix', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Basis State')
axes[0].set_ylabel('Basis State')

# Eigenvalue spectrum
axes[1].plot(eigenvalues, 'o-', linewidth=2, markersize=8)
axes[1].axhline(eigenvalues[0], color='red', linestyle='--', alpha=0.5, label='Ground state')
axes[1].set_xlabel('Eigenvalue Index', fontsize=12)
axes[1].set_ylabel('Energy', fontsize=12)
axes[1].set_title('Energy Spectrum', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print('✅ Hamiltonian visualized')

## 3. VQE Optimization

Optimize quantum circuit parameters to minimize energy.

In [ ]:
# Setup optimization
coords = torch.randn(n_qubits, 3)
optimizer = torch.optim.Adam([solver.theta], lr=0.05)

# Optimization loop
n_iterations = 50
energies = []

print('🎯 Optimizing VQE parameters...')
for i in range(n_iterations):
    optimizer.zero_grad()
    energy = solver(coords)
    energy.backward()
    optimizer.step()
    energies.append(energy.item())
    
    if (i+1) % 10 == 0:
        print(f'Iteration {i+1}: Energy = {energy.item():.4f}')

print(f'\n✅ Optimization complete')
print(f'Initial energy: {energies[0]:.4f}')
print(f'Final energy: {energies[-1]:.4f}')
print(f'Improvement: {energies[0] - energies[-1]:.4f} ({(energies[0] - energies[-1])/abs(energies[0])*100:.1f}%)')

# Plot convergence
plt.figure(figsize=(10, 6))
plt.plot(energies, linewidth=2)
plt.xlabel('Iteration', fontsize=12, fontweight='bold')
plt.ylabel('Energy', fontsize=12, fontweight='bold')
plt.title('VQE Convergence', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Gradient Analysis

Examine parameter gradients during optimization.

In [ ]:
# Compute gradients
solver.theta.requires_grad = True
energy = solver(coords)
energy.backward()

grads = solver.theta.grad.detach().numpy()

print(f'Gradient statistics:')
print(f'  Mean: {np.mean(grads):.6f}')
print(f'  Std: {np.std(grads):.6f}')
print(f'  Max: {np.max(np.abs(grads)):.6f}')

# Visualize
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(grads, 'o-', alpha=0.6)
plt.axhline(0, color='red', linestyle='--', alpha=0.5)
plt.xlabel('Parameter Index', fontsize=12)
plt.ylabel('Gradient', fontsize=12)
plt.title('Parameter Gradients', fontsize=12, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(grads, bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Gradient Value', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Gradient Distribution', fontsize=12, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print('✅ Gradient analysis complete')

## 🎓 Summary

You've explored the quantum computing aspects:

- Built VQE ansatz circuits with parameterized gates
- Analyzed Hamiltonian construction and eigenspectra
- Optimized parameters using gradient descent
- Examined gradient flow through quantum circuits

**Key insight:** VQE enables differentiable quantum energy evaluation!